In [1]:
import pandas as pd
import numpy as np 
data = pd.read_csv('training_data.csv')  

In [2]:
#data['PEoPhi'] = data['PE']/data['PHIND']
data['ILDoPhi']  = data['ILD_log10']/data['PHIND']

data.head(10)

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,ILDoPhi
0,3,A1 SH,SHRIMPLIN,2793.0,77.45,0.664,9.9,11.915,4.6,1,1.000,0.055728
1,3,A1 SH,SHRIMPLIN,2793.5,78.26,0.661,14.2,12.565,4.1,1,0.979,0.052606
2,3,A1 SH,SHRIMPLIN,2794.0,79.05,0.658,14.8,13.050,3.6,1,0.957,0.050421
3,3,A1 SH,SHRIMPLIN,2794.5,86.10,0.655,13.9,13.115,3.5,1,0.936,0.049943
4,3,A1 SH,SHRIMPLIN,2795.0,74.58,0.647,13.5,13.300,3.4,1,0.915,0.048647
5,3,A1 SH,SHRIMPLIN,2795.5,73.97,0.636,14.0,13.385,3.6,1,0.894,0.047516
6,3,A1 SH,SHRIMPLIN,2796.0,73.72,0.630,15.6,13.930,3.7,1,0.872,0.045226
7,3,A1 SH,SHRIMPLIN,2796.5,75.65,0.625,16.5,13.920,3.5,1,0.830,0.044899
8,3,A1 SH,SHRIMPLIN,2797.0,73.79,0.624,16.2,13.980,3.4,1,0.809,0.044635
9,3,A1 SH,SHRIMPLIN,2797.5,76.89,0.615,16.9,14.220,3.5,1,0.787,0.043249


In [3]:
data_test = data[data['Well Name'] == 'SHANKLE']
data = data[data['Well Name'] != 'SHANKLE']
#data2 = data.sample(n=729)
#data2.describe()   # use random sampled 729 events gives 
#F1 scroe 0.51 for CV(nonstratified split), 0.36 for Test 
#F1 scroe 0.55 for CV(stratified split), 0.41 for Test 

#data['BoundaryFlag'] = pd.Series(0,index=data.index)  # add one column. 
#previousface=-1
#for index, row in data.iterrows():
#    if row['Facies']!=previousface: 
#        data.at[index, 'BoundaryFlag'] =  1   
#        data.at[index-1, 'BoundaryFlag'] =  1    
#        data.set_value(index, 'BoundaryFlag',1) ##old version
#        previousface = row['Facies']
#data = data[data['BoundaryFlag']==0]
#data= data.drop(['BoundaryFlag'],axis=1)
#data.describe
#w/o boundary in data, F1 score 0.61 for CV and 0.43 for Test 
#w boundary in data, F1 score 0.50 for CV and 0.55 for Test

In [4]:
features = ['GR', 'ILD_log10', 'DeltaPHI','PHIND','PE','NM_M', 'RELPOS','ILDoPhi']
feature_vectors = data[features]
facies_labels = data['Facies']

feature_vectors_test = data_test[features]
facies_labels_test = data_test['Facies']

facies_labels.describe()
# Store well labels and depths
well = data['Well Name'].values
depth = data['Depth'].values
well_test = data_test['Well Name'].values
depth_test = data_test['Depth'].values
# Fill 'PE' missing values with mean
from sklearn import preprocessing
imp = preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(feature_vectors)
feature_vectors = imp.transform(feature_vectors)
imp.fit(feature_vectors_test)
feature_vectors_test = imp.transform(feature_vectors_test)


In [5]:



# Feature windows concatenation function
def augment_features_window(X, N_neig):
    
    # Parameters
    N_row = X.shape[0]
    N_feat = X.shape[1]

    # Zero padding
    X = np.vstack((np.zeros((N_neig, N_feat)), X, (np.zeros((N_neig, N_feat)))))

    # Loop over windows
    X_aug = np.zeros((N_row, N_feat*(2*N_neig+1)))
    for r in np.arange(N_row)+N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            this_row = np.hstack((this_row, X[r+c]))
        X_aug[r-N_neig] = this_row

    return X_aug


# Feature gradient computation function
def augment_features_gradient(X, depth):
    
    # Compute features gradient
    d_diff = np.diff(depth).reshape((-1, 1))
    d_diff[d_diff==0] = 0.001
    X_diff = np.diff(X, axis=0)
    X_grad = X_diff / d_diff
        
    # Compensate for last missing value
    X_grad = np.concatenate((X_grad, np.zeros((1, X_grad.shape[1]))))
    
    return X_grad


# Feature augmentation function
def augment_features(X, well, depth, N_neig=1):
    
    # Augment features
    X_aug = np.zeros((X.shape[0], X.shape[1]*(N_neig*2+2)))
    for w in np.unique(well):
        w_idx = np.where(well == w)[0]
        X_aug_win = augment_features_window(X[w_idx, :], N_neig)
        X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
        X_aug[w_idx, :] = np.concatenate((X_aug_win, X_aug_grad), axis=1)
    
    # Find padded rows
    padded_rows = np.unique(np.where(X_aug[:, 0:7] == np.zeros((1, 7)))[0])
    
    return X_aug, padded_rows

In [6]:
X_aug, padded_rows = augment_features(feature_vectors, well, depth)
X_aug_test, padded_rows_test = augment_features(feature_vectors_test, well_test, depth_test)

X_aug.shape

(2783, 32)

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_aug)
scaled_features = scaler.transform(X_aug) #ndarray now. 
scaler_test = StandardScaler().fit(X_aug_test)
scaled_features_test = scaler.transform(X_aug_test) #ndarray now. 
#scaled_features.shape
#scaler = StandardScaler().fit(feature_vectors)
#scaled_features = scaler.transform(feature_vectors) #ndarray now. 

#df3 = pd.DataFrame(np.random.randn(10, 5),columns=['a', 'b', 'c', 'd', 'e'])
#df3.describe
scaled_features.shape

(2783, 32)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(scaled_features, facies_labels,test_size=0.05, random_state=42,stratify=facies_labels)
#X_train, X_cv, y_train, y_cv = train_test_split(scaled_features, facies_labels,test_size=0.05, random_state=42)
#y_train.value_counts()
X_train.shape

(2643, 32)

In [9]:
from sklearn.metrics import classification_report
target_names = ['SS', 'CSiS', 'FSiS', 'SiSh','MS', 'WS', 'D','PS', 'BS']

In [10]:
from xgboost.sklearn import  XGBClassifier
clf = XGBClassifier(learning_rate=0.12,max_depth=3,min_child_weight=10,n_estimators=150,seed=100,colsample_bytree=0.9)
#from sklearn.neural_network import MLPClassifier
#clf = MLPClassifier(solver='lbfgs', alpha=1e-1,hidden_layer_sizes=(3), random_state=1)
clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, gamma=0, learning_rate=0.12, max_delta_step=0,
       max_depth=3, min_child_weight=10, missing=None, n_estimators=150,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=100,
       silent=True, subsample=1)

In [11]:
y_cv_pred = clf.predict(X_cv) 
print(classification_report(y_cv, y_cv_pred,target_names=target_names))

             precision    recall  f1-score   support

         SS       0.88      0.88      0.88         8
       CSiS       0.76      0.85      0.80        33
       FSiS       0.82      0.72      0.77        25
       SiSh       0.50      0.56      0.53         9
         MS       0.56      0.50      0.53        10
         WS       0.74      0.70      0.72        20
          D       1.00      0.75      0.86         4
         PS       0.60      0.65      0.63        23
         BS       0.86      0.75      0.80         8

avg / total       0.73      0.72      0.72       140



C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [12]:
y_pred = clf.predict(scaled_features_test)

C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [13]:

print(classification_report(facies_labels_test, y_pred,target_names=target_names))

             precision    recall  f1-score   support

         SS       0.82      0.42      0.55        89
       CSiS       0.42      0.83      0.56        89
       FSiS       0.87      0.56      0.68       117
       SiSh       0.16      0.43      0.23         7
         MS       0.00      0.00      0.00        19
         WS       0.73      0.79      0.76        71
          D       1.00      0.65      0.79        17
         PS       0.71      0.75      0.73        40

avg / total       0.69      0.61      0.62       449



C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 8, does not match size of target_names, 9
  .format(len(labels), len(target_names))
